In [1]:
import os
import pandas as pd
import numpy as np
import openpyxl as ox

from openpyxl import load_workbook
from openpyxl import workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell.cell import WriteOnlyCell

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

import string

import pyodbc

print("Артём обновляет отчёт по зачислению...")

Артём обновляет отчёт по зачислению...


In [2]:
data = pd.read_excel(r'\\synergy.local\Documents\11.Коммерческий департамент\13.Отдел прямых продаж\Зачисление\Для Альбины.xlsx', 
                     sheet_name='Для заполнения')

In [3]:
data = data.loc[data['Дата оплаты'] != 'Дубль']
data = data.drop(columns=['№', 'Договор', "Абитуриент", "Встреча",
                          "Менеджер", "Визуалы", "Дата звонка", "Дата доплаты", 
                          "ДАТА ПОДАЧИ", "Сумма доплаты", "STAGE_DO.Статус", "STAGE_OPP.Статус",
                          "STAGE_MOI.Статус", "Комментарий", "Дата зачисления план", 
                          "Дата зачисления факт", "Типовой статус РОП", "Комментарий РОП",
                          "Дата зачисления план РОП", "Дата зачисления факт РОП", "архив",
                          "Ответсвенный ГО", "Дата первого звонка", "Дата следующего звонка",
                          "Успешный старт"])

In [12]:
data['Год'] = data['Дата оплаты'].dt.year
data['Месяц'] = data['Дата оплаты'].dt.month
data[['Год', 'Месяц']]= data[['Год', 'Месяц']].fillna(0)
data[['Год', 'Месяц']] =data[['Год', 'Месяц']].astype(int)
data = data.loc[data['Дата оплаты'] >= "01.01.2021"]
data['Категория клиента'] = data['Категория клиента'].replace(' 5 Д. НД', '')
data = data.loc[data['Категория клиента'] != 'Абитуриент оформил заявление на возврат ДС']
data = data.loc[data['Категория клиента'] != 'абитуриент сообщил о желании оформить возврат ДС']
data = data.loc[data['Категория клиента'] != 'Абитуриент сообщил о желании оформить возврат ДС']
data['Категория клиента'] = data['Категория клиента'].replace('неполный', 'Неполный')

data.loc[data['Категория клиента'] == "Зачислен", ['Статус']] = 1
data.loc[data['Категория клиента'] == "Зачислен без ПД", ['Статус']] = 1
data.loc[data['Категория клиента'] == "Полный ПД", ['Статус']] = 1
data.loc[data['Статус'] != 1, ['Статус']] = 0
data['Статус'] = data['Статус'].astype(int)
data

,ID сделки,ОП,КЦ,Дата оплаты,Чистая,УЗ,Категория клиента,Статус зачисления,Статус передачи в работу АМ,Год,Месяц,Статус
18736,3720668,ОП 5,КЦ 1,2021-01-05,NaN,МФПУ,Зачислен,NaN,В работе АМ,2021,1,1
18737,3719757,ОП 5,КЦ 1,2021-01-01,NaN,МФПУ,Зачислен,NaN,В работе АМ,2021,1,1
18739,3722130,ОП 1,КЦ 1,2021-01-02,ДО,МОИ,Зачислен,,В работе АМ,2021,1,1
18740,3687215,5 ЯР,КЦ 3,2021-01-03,NaN,МФПУ,Зачислен,NaN,В работе АМ,2021,1,1
18741,3722215,ОП 4,КЦ 1,2021-01-03,ДО,МФПУ,Зачислен,NaN,В работе АМ,2021,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
80051,5675411,ОП 5,КЦ 1,2023-06-12,NaN,Колледж,NaN,NaN,NaN,2023,6,0
80052,5691152,ОП 16,КЦ 2,2023-06-12,NaN,Бакалавриат,NaN,NaN,NaN,2023,6,0
80053,5693483,3 ВР,КЦ 4,2023-06-12,NaN,Курсы,Зачислен,NaN,В работе АМ,2023,6,1
80054,5686623,ОП 10.2,КЦ 1,2023-06-12,NaN,Колледж,NaN,NaN,NaN,2023,6,0


In [15]:
svod_23_1 = data.query('Год == 2023 & УЗ != "МТИ"')
svod_23_1 = svod_23.pivot_table(index='Месяц',
                              columns='Категория клиента',
                              values='ID сделки',
                              aggfunc='count')
svod_23_1

Категория клиента,Выпуск будущего периода 2022,Готов к зачислению,Зачислен,Зачислен,Зачислен без ПД,Недоплата,Неполный пакет документов,Нет личного дела,выпуск будущего периода 2022
Месяц,,,,,,,,,
1,59.0,11.0,405.0,NaN,783.0,NaN,NaN,6.0,NaN
2,90.0,36.0,309.0,NaN,709.0,NaN,2.0,5.0,NaN
3,170.0,89.0,398.0,NaN,845.0,NaN,4.0,9.0,NaN
4,187.0,244.0,502.0,4.0,688.0,1.0,4.0,10.0,1.0
5,294.0,431.0,514.0,NaN,586.0,6.0,70.0,96.0,NaN
6,30.0,57.0,127.0,NaN,57.0,2.0,102.0,69.0,NaN


In [26]:
svod_23_2 = data.query('Год == 2023 & УЗ == "МФПУ" | УЗ == "МФПУ//ДПО"')
svod_23_2 = svod_23_2.loc[((svod_23_2['Категория клиента'] == 'Зачислен без ПД') | (svod_23_2['Категория клиента'] == np.nan))]
svod_23_2 = svod_23_2.pivot_table(index='Месяц',
                              columns='Статус зачисления',
                              values='ID сделки',
                              aggfunc='count')
svod_23_2

Статус зачисления,Минимальный ПД,Минимальный пд,Отток,Полный пакет документов
Месяц,,,,
1,2.0,1.0,3.0,715.0
2,5.0,NaN,1.0,627.0
3,7.0,NaN,1.0,666.0
4,28.0,NaN,NaN,217.0
5,129.0,NaN,1.0,88.0
6,2.0,NaN,NaN,NaN


In [28]:
svod_23_3 = data.query('Год == 2023 & УЗ == "МФПУ" | УЗ == "МФПУ//ДПО"')
svod_23_3 = svod_23_3.loc[((svod_23_3['Статус зачисления'] == 'Оригиналы документов собраны') | (svod_23_3['Статус зачисления'] == 'Полный пакет документов'))]
svod_23_3 = svod_23_3.pivot_table(index='Месяц',
                              values='ID сделки',
                              aggfunc='count')
svod_23_3

,ID сделки
Месяц,
1,715
2,627
3,666
4,217
5,88


In [29]:
svod_23_4 = data.query('Год == 2023 & УЗ == "МФПУ" | УЗ == "МФПУ//ДПО"')
svod_23_4 = svod_23_4.loc[data['Категория клиента'] != 'Зачислен']
svod_23_4 = svod_23_4.loc[data['Категория клиента'] != 'Зачислен без ПД']
svod_23_4 = svod_23_4.loc[data['Категория клиента'] != 'Дубль']
svod_23_4 = svod_23_4.loc[data['Категория клиента'] != 'Готов к зачислению']
svod_23_4 = svod_23_4.pivot_table(index='Месяц',
                              values='ID сделки',
                              aggfunc='count')
svod_23_4

,ID сделки
Месяц,
1,53
2,66
3,86
4,54
5,17


In [81]:
def svod(year):
    x_1 = 0
    x_2 = 0
    x_3 = 0
    x_4 = 0
    
    x_1 = data.loc[((data['Год'] == year) & (data['УЗ'] != "МТИ"))] 
#    svod_1 = data.query('Год == year & УЗ != "МТИ"')
    x_1 = x_1.pivot_table(index='Месяц',
                              columns='Категория клиента',
                              values='ID сделки',
                              aggfunc='count')
    
    x_2 = data.loc[((data['Год'] == year) & ((data['УЗ'] != "МФПУ") | (data['УЗ'] != "МФПУ//ДПО")))] 
#    svod_2 = data.query('Год == year & УЗ == "МФПУ" | УЗ == "МФПУ//ДПО"')
    x_2 = x_2.loc[((x_2['Категория клиента'] == 'Зачислен без ПД') | (x_2['Категория клиента'] == np.nan))]
    x_2 = x_2.pivot_table(index='Месяц',
                                  columns='Статус зачисления',
                                  values='ID сделки',
                                  aggfunc='count')
    
    x_3 = data.loc[((data['Год'] == year) & ((data['УЗ'] != "МФПУ") | (data['УЗ'] != "МФПУ//ДПО")))] 
#    svod_3 = data.query('Год == year & УЗ == "МФПУ" | УЗ == "МФПУ//ДПО"')
    x_3 = x_3.loc[((x_3['Статус зачисления'] == 'Оригиналы документов собраны') | (x_3['Статус зачисления'] == 'Полный пакет документов'))]
    x_3 = x_3.pivot_table(index='Месяц',
                                  values='ID сделки',
                                  aggfunc='count')
    
    x_4 = data.loc[((data['Год'] == year) & ((data['УЗ'] != "МФПУ") | (data['УЗ'] != "МФПУ//ДПО")))] 
#    svod_4 = data.query('Год == year & УЗ == "МФПУ" | УЗ == "МФПУ//ДПО"')
    x_4 = x_4.loc[x_4['Категория клиента'] != 'Зачислен']
    x_4 = x_4.loc[x_4['Категория клиента'] != 'Зачислен без ПД']
    x_4 = x_4.loc[x_4['Категория клиента'] != 'Дубль']
    x_4 = x_4.loc[x_4['Категория клиента'] != 'Готов к зачислению']
    x_4 = x_4.pivot_table(index='Месяц',
                                  values='ID сделки',
                                  aggfunc='count')
    
    x = x_1.merge(x_2, on='Месяц', how='outer')
    x = x.merge(x_3, on='Месяц', how='outer')
    x = x.merge(x_4, on='Месяц', how='outer')
    
    return x

In [86]:
svod(2023)

,Выпуск будущего периода 2022,Готов к зачислению,Зачислен,Зачислен,Зачислен без ПД,Недоплата,Неполный пакет документов,Нет личного дела,выпуск будущего периода 2022,Минимальный ПД,Минимальный пд,Минимльный ПД,Отток,Полный пакет документов,полный пакет документов,ID сделки_x,ID сделки_y
Месяц,,,,,,,,,,,,,,,,,
1,59.0,11.0,405.0,NaN,783.0,NaN,NaN,6.0,NaN,2.0,1.0,NaN,4.0,776.0,NaN,776.0,65
2,90.0,36.0,309.0,NaN,709.0,NaN,2.0,5.0,NaN,6.0,NaN,NaN,1.0,701.0,1.0,701.0,97
3,170.0,89.0,398.0,NaN,845.0,NaN,4.0,9.0,NaN,15.0,1.0,NaN,1.0,828.0,NaN,828.0,183
4,187.0,244.0,502.0,4.0,688.0,1.0,4.0,10.0,1.0,114.0,NaN,NaN,2.0,572.0,NaN,572.0,207
5,294.0,431.0,514.0,NaN,586.0,6.0,70.0,96.0,NaN,445.0,NaN,10.0,2.0,129.0,NaN,129.0,466
6,30.0,57.0,127.0,NaN,57.0,2.0,102.0,69.0,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,362
